# Model Development
- Train/Test Split > Tuning > Cross Verification > Training > Testing > Model Saved
- Traing Loss and Training AUC Plot and Test ROC plot

In [ ]:
# combined_transformer_gcn_pipeline_optuna_cv.py
import os
import json
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.data import DataLoader, Data
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import rdMolDescriptors
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import (
    roc_auc_score, accuracy_score, matthews_corrcoef,
    cohen_kappa_score, brier_score_loss, confusion_matrix,
    precision_score, f1_score, roc_curve
)
import matplotlib.pyplot as plt
import optuna

# Ensure output directory
output_dir = 'Transformer_GCN'
os.makedirs(output_dir, exist_ok=True)

# Expanded atom features
def atom_features(atom):
    return torch.tensor([
        # Basic properties
        atom.GetAtomicNum(),                     # Atomic number
        atom.GetDegree(),                        # Number of bonded neighbors
        atom.GetFormalCharge(),                  # Formal charge
        atom.GetNumRadicalElectrons(),           # Number of radical electrons
        int(atom.GetIsAromatic()),               # Aromaticity flag
        
        # Extended properties
        atom.GetExplicitValence(),               # Explicit valence
        atom.GetImplicitValence(),               # Implicit valence
        atom.GetTotalValence(),                  # Total valence
        atom.GetNumImplicitHs(),                 # Number of implicit hydrogens
        atom.GetHybridization(),                 # Hybridization state
        atom.GetTotalNumHs(),                    # Total number of hydrogens
        
        # Topological properties
        int(atom.IsInRing()),                    # Whether the atom is in a ring
        int(atom.IsInRingSize(3)),               # Whether in 3-membered ring
        int(atom.IsInRingSize(4)),               # Whether in 4-membered ring
        int(atom.IsInRingSize(5)),               # Whether in 5-membered ring
        int(atom.IsInRingSize(6)),               # Whether in 6-membered ring
        int(atom.IsInRingSize(7)),               # Whether in 7-membered ring
        
        # Electronic properties
        atom.GetChiralTag(),                     # Chirality
        atom.GetMass(),                          # Atomic mass
        Chem.rdMolDescriptors.CalcCrippenDescriptors(
            Chem.MolFromSmiles(f"[{atom.GetSymbol()}]"))[0]  # LogP contribution
    ], dtype=torch.float)

# Molecule to PyTorch Geometric graph
def mol_to_graph(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None: return None
    atoms = [atom_features(atom) for atom in mol.GetAtoms()]
    if not atoms: return None
    x = torch.stack(atoms, dim=0)
    edge_index = []
    for bond in mol.GetBonds():
        i, j = bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()
        edge_index += [[i, j], [j, i]]
    edge_index = torch.tensor(edge_index).t().contiguous()
    edge_attr = [[bond.GetBondTypeAsDouble(), bond.GetIsConjugated(), bond.IsInRing()] * 2 for bond in mol.GetBonds()]
    edge_attr = torch.tensor(edge_attr, dtype=torch.float).reshape(-1, 3)
    return Data(x=x, edge_index=edge_index, edge_attr=edge_attr)

# Load dataset
def load_dataset(path, smiles_col='Smiles', target_col='Target'):
    df = pd.read_excel(path)
    graphs = []
    for _, row in df.iterrows():
        if pd.isna(row[smiles_col]) or pd.isna(row[target_col]):
            continue
        g = mol_to_graph(str(row[smiles_col]))
        if g is not None:
            g.y = torch.tensor([row[target_col]], dtype=torch.float)
            g.smiles = row[smiles_col]
            graphs.append(g)
    return graphs

# MultiHeadAttention Module
class MultiHeadAttention(torch.nn.Module):
    def __init__(self, hidden_dim, num_heads, dropout=0.1):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.num_heads = num_heads
        
        # Ensure hidden_dim is divisible by num_heads
        # If not, we'll adjust the head_dim to make it work
        self.head_dim = hidden_dim // num_heads
        
        # Actual dimension after ensuring divisibility
        self.actual_dim = self.head_dim * num_heads
        
        self.query = torch.nn.Linear(hidden_dim, self.actual_dim)
        self.key = torch.nn.Linear(hidden_dim, self.actual_dim)
        self.value = torch.nn.Linear(hidden_dim, self.actual_dim)
        self.proj = torch.nn.Linear(self.actual_dim, hidden_dim)
        self.dropout = torch.nn.Dropout(dropout)
        
    def forward(self, x):
        batch_size = x.size(0)
        seq_length = x.size(1)
        
        # Linear projections
        q = self.query(x).view(batch_size, seq_length, self.num_heads, self.head_dim).transpose(1, 2)
        k = self.key(x).view(batch_size, seq_length, self.num_heads, self.head_dim).transpose(1, 2)
        v = self.value(x).view(batch_size, seq_length, self.num_heads, self.head_dim).transpose(1, 2)
        
        # Scaled dot-product attention
        scores = torch.matmul(q, k.transpose(-2, -1)) / (self.head_dim ** 0.5)
        
        # Apply attention weights
        attn_weights = F.softmax(scores, dim=-1)
        attn_weights = self.dropout(attn_weights)
        context = torch.matmul(attn_weights, v)
        
        # Concatenate heads and put through final linear layer
        context = context.transpose(1, 2).contiguous().view(batch_size, seq_length, self.actual_dim)
        output = self.proj(context)
        
        return output

# Transformer Block
class TransformerBlock(torch.nn.Module):
    def __init__(self, hidden_dim, num_heads, ff_dim, dropout=0.1):
        super().__init__()
        self.attention = MultiHeadAttention(hidden_dim, num_heads, dropout)
        self.norm1 = torch.nn.LayerNorm(hidden_dim)
        self.norm2 = torch.nn.LayerNorm(hidden_dim)
        self.ff = torch.nn.Sequential(
            torch.nn.Linear(hidden_dim, ff_dim),
            torch.nn.GELU(),
            torch.nn.Dropout(dropout),
            torch.nn.Linear(ff_dim, hidden_dim),
            torch.nn.Dropout(dropout)
        )
        
    def forward(self, x):
        # Self attention + residual connection
        attn_output = self.attention(x)
        x = self.norm1(x + attn_output)
        
        # Feed forward + residual connection
        ff_output = self.ff(x)
        x = self.norm2(x + ff_output)
        
        return x

# TransformerGCN model
class TransformerGCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers=3, 
                 num_heads=4, ff_dim_factor=4, dropout=0.1):
        super().__init__()
        self.embedding = torch.nn.Linear(in_channels, hidden_channels)
        
        # GCN Layers
        self.convs = torch.nn.ModuleList()
        for _ in range(num_layers):
            self.convs.append(GCNConv(hidden_channels, hidden_channels))
        
        # Transformer blocks (one after each GCN layer)
        self.transformers = torch.nn.ModuleList()
        for _ in range(num_layers):
            self.transformers.append(
                TransformerBlock(hidden_channels, num_heads, hidden_channels * ff_dim_factor, dropout)
            )
        
        # Output layers
        self.fc1 = torch.nn.Linear(hidden_channels, hidden_channels // 2)
        self.fc2 = torch.nn.Linear(hidden_channels // 2, out_channels)
        self.dropout = dropout
        
        # Save for reshaping in forward
        self.hidden_channels = hidden_channels
        self.num_layers = num_layers

    def forward(self, x, edge_index, edge_attr, batch):
        # Initial embedding
        x = self.embedding(x)
        
        # Apply GCN and Transformer layers
        for i, (conv, transformer) in enumerate(zip(self.convs, self.transformers)):
            # GCN Layer
            x_conv = conv(x, edge_index)
            x_conv = F.relu(x_conv)
            x_conv = F.dropout(x_conv, p=self.dropout, training=self.training)
            
            # Reshape for transformer (using a safer approach)
            # Create a padded tensor per batch
            batch_idx = torch.unique(batch)
            transformed_features = []
            
            for b in batch_idx:
                # Get nodes for this batch
                mask = (batch == b)
                nodes_batch = x_conv[mask]
                
                # Process through transformer (add batch dimension)
                if nodes_batch.size(0) > 0:  # Skip empty batches
                    # Add batch and sequence dimensions
                    nodes_batch = nodes_batch.unsqueeze(0)
                    
                    # Apply transformer
                    transformed = transformer(nodes_batch)
                    
                    # Remove batch dimension
                    transformed = transformed.squeeze(0)
                    transformed_features.append(transformed)
            
            # Check if we have any valid features
            if transformed_features:
                # Concatenate all transformed features
                x = torch.cat(transformed_features, dim=0)
            else:
                # Fallback - shouldn't typically happen
                x = x_conv
        
        # Global pooling
        x = global_mean_pool(x, batch)
        
        # MLP for final prediction
        x = self.fc1(x)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.fc2(x)
        
        return x

# Training and AUC
def train(model, loader, optimizer, criterion):
    model.train()
    total_loss = 0
    for data in loader:
        optimizer.zero_grad()
        out = model(data.x, data.edge_index, data.edge_attr, data.batch).squeeze()
        loss = criterion(torch.sigmoid(out), data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

def compute_auc(model, loader):
    model.eval()
    y_true, y_probs = [], []
    with torch.no_grad():
        for data in loader:
            out = model(data.x, data.edge_index, data.edge_attr, data.batch).squeeze()
            y_probs.extend(torch.sigmoid(out).cpu().numpy())
            y_true.extend(data.y.cpu().numpy())
    return roc_auc_score(y_true, y_probs)

# Model evaluation
def get_metrics(model, loader):
    model.eval()
    y_true, y_probs = [], []
    with torch.no_grad():
        for data in loader:
            out = model(data.x, data.edge_index, data.edge_attr, data.batch).squeeze()
            y_probs.extend(torch.sigmoid(out).cpu().numpy())
            y_true.extend(data.y.cpu().numpy())
    y_preds = (np.array(y_probs) > 0.5).astype(int)
    return {
        'AUC': roc_auc_score(y_true, y_probs),
        'Accuracy': accuracy_score(y_true, y_preds),
        'MCC': matthews_corrcoef(y_true, y_preds),
        'Precision': precision_score(y_true, y_preds),
        'F1': f1_score(y_true, y_preds),
        'Kappa': cohen_kappa_score(y_true, y_preds),
        'Brier': brier_score_loss(y_true, y_probs),
    }

# Globals for Optuna
trial_train_loader = None
trial_val_loader = None

# Optuna objective
def objective(trial):
    # Ensure hidden_channels is divisible by all possible num_heads values
    hidden_channels = trial.suggest_int("hidden_channels", 32, 128, step=16)  # Ensures divisibility
    num_layers = trial.suggest_int("num_layers", 2, 4)
    num_heads = trial.suggest_int("num_heads", 2, 8, step=2)  # Use only even numbers
    ff_dim_factor = trial.suggest_int("ff_dim_factor", 2, 4)
    dropout = trial.suggest_float("dropout", 0.1, 0.5)
    lr = trial.suggest_float("lr", 1e-4, 1e-2, log=True)

    model = TransformerGCN(20, hidden_channels, 1, num_layers, num_heads, ff_dim_factor, dropout)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = torch.nn.BCELoss()

    for epoch in range(10):
        train(model, trial_train_loader, optimizer, criterion)

    return compute_auc(model, trial_val_loader)

def main():
    global trial_train_loader, trial_val_loader

    graphs = load_dataset("dataset_main.xlsx")
    train_data, test_data = train_test_split(graphs, test_size=0.2, random_state=42)
    trial_train, trial_val = train_test_split(train_data, test_size=0.2, random_state=42)
    trial_train_loader = DataLoader(trial_train, batch_size=32, shuffle=True)
    trial_val_loader = DataLoader(trial_val, batch_size=32)

    # Optuna tuning
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=30)
    best_params = study.best_trial.params
    json.dump(best_params, open(os.path.join(output_dir, "study_best_params_transformer_gcn.json"), "w"), indent=4)

    # 10-fold cross-validation (mean metrics only)
    kf = KFold(n_splits=10, shuffle=True, random_state=42)
    metrics_list = []

    for fold, (train_idx, val_idx) in enumerate(kf.split(train_data), start=1):
        model = TransformerGCN(
            20, best_params["hidden_channels"], 1, 
            best_params["num_layers"], best_params["num_heads"],
            best_params["ff_dim_factor"], best_params["dropout"]
        )
        optimizer = torch.optim.Adam(model.parameters(), lr=best_params["lr"])
        criterion = torch.nn.BCELoss()

        train_fold = [train_data[i] for i in train_idx]
        val_fold = [train_data[i] for i in val_idx]
        loader_train = DataLoader(train_fold, batch_size=32, shuffle=True)
        loader_val = DataLoader(val_fold, batch_size=32)

        for epoch in range(30):
            train(model, loader_train, optimizer, criterion)

        metrics = get_metrics(model, loader_val)
        metrics_list.append(metrics)

    # Save and print mean of cross-validation metrics
    df_cv = pd.DataFrame(metrics_list)
    mean_metrics = df_cv.mean()
    df_cv.loc['mean'] = mean_metrics
    df_cv.tail(1).to_excel(os.path.join(output_dir, "cv_metrics_mean.xlsx"), index=False)

    print("\n📊 10-Fold Cross-Validation Mean Metrics:")
    for key, value in mean_metrics.items():
        print(f"{key}: {value:.4f}")

    # Final training on full training set
    full_train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_data, batch_size=32)

    final_model = TransformerGCN(
        20, best_params["hidden_channels"], 1, 
        best_params["num_layers"], best_params["num_heads"],
        best_params["ff_dim_factor"], best_params["dropout"]
    )
    optimizer = torch.optim.Adam(final_model.parameters(), lr=best_params["lr"])
    criterion = torch.nn.BCELoss()

    losses, aucs = [], []
    for epoch in range(1, 101):
        loss = train(final_model, full_train_loader, optimizer, criterion)
        auc = compute_auc(final_model, full_train_loader)
        losses.append(loss)
        aucs.append(auc)
        print(f"[Final Train] Epoch {epoch} | Loss: {loss:.4f} | AUC: {auc:.4f}")

    torch.save(final_model.state_dict(), os.path.join(output_dir, "transformer_gcn_best_model.pth"))

    plt.figure()
    plt.plot(losses, label="Loss")
    plt.title("Final Training Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.savefig(os.path.join(output_dir, "final_training_loss.png"))
    plt.close()

    plt.figure()
    plt.plot(aucs, label="AUC", color='red')
    plt.title("Final Training AUC")
    plt.xlabel("Epoch")
    plt.ylabel("AUC")
    plt.savefig(os.path.join(output_dir, "final_training_auc.png"))
    plt.close()

    # Final test set evaluation
    test_metrics = get_metrics(final_model, test_loader)
    pd.DataFrame([test_metrics]).to_excel(os.path.join(output_dir, "test_metrics.xlsx"), index=False)

    print("\n🧪 Final Test Set Metrics:")
    for key, value in test_metrics.items():
        print(f"{key}: {value:.4f}")

    # ROC Curve
    model = final_model
    model.eval()
    y_true, y_probs = [], []
    with torch.no_grad():
        for data in test_loader:
            out = model(data.x, data.edge_index, data.edge_attr, data.batch).squeeze()
            y_probs.extend(torch.sigmoid(out).cpu().numpy())
            y_true.extend(data.y.cpu().numpy())
    fpr, tpr, _ = roc_curve(y_true, y_probs)
    plt.figure()
    plt.plot(fpr, tpr, label=f"AUC = {test_metrics['AUC']:.2f}")
    plt.plot([0, 1], [0, 1], '--', color='gray')
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("Test ROC Curve")
    plt.legend()
    plt.savefig(os.path.join(output_dir, "test_roc.png"))
    plt.close()

if __name__ == "__main__":
    main()

In [ ]:
{'hidden_channels': 80, 'num_layers': 2, 'num_heads': 8, 'ff_dim_factor': 4, 'dropout': 0.22570405839644114, 'lr': 0.00010100422122258602}

## Confusion matrix generation

In [ ]:
import os
import json
import torch
import torch.nn.functional as F
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from torch_geometric.data import DataLoader, Data
from torch_geometric.nn import GCNConv, global_mean_pool
from rdkit import Chem
from rdkit.Chem import rdMolDescriptors

# Reuse atom features function from cell1
def atom_features(atom):
    return torch.tensor([
        # Basic properties
        atom.GetAtomicNum(),                     # Atomic number
        atom.GetDegree(),                        # Number of bonded neighbors
        atom.GetFormalCharge(),                  # Formal charge
        atom.GetNumRadicalElectrons(),           # Number of radical electrons
        int(atom.GetIsAromatic()),               # Aromaticity flag
        
        # Extended properties
        atom.GetExplicitValence(),               # Explicit valence
        atom.GetImplicitValence(),               # Implicit valence
        atom.GetTotalValence(),                  # Total valence
        atom.GetNumImplicitHs(),                 # Number of implicit hydrogens
        atom.GetHybridization(),                 # Hybridization state
        atom.GetTotalNumHs(),                    # Total number of hydrogens
        
        # Topological properties
        int(atom.IsInRing()),                    # Whether the atom is in a ring
        int(atom.IsInRingSize(3)),               # Whether in 3-membered ring
        int(atom.IsInRingSize(4)),               # Whether in 4-membered ring
        int(atom.IsInRingSize(5)),               # Whether in 5-membered ring
        int(atom.IsInRingSize(6)),               # Whether in 6-membered ring
        int(atom.IsInRingSize(7)),               # Whether in 7-membered ring
        
        # Electronic properties
        atom.GetChiralTag(),                     # Chirality
        atom.GetMass(),                          # Atomic mass
        Chem.rdMolDescriptors.CalcCrippenDescriptors(
            Chem.MolFromSmiles(f"[{atom.GetSymbol()}]"))[0]  # LogP contribution
    ], dtype=torch.float)

def mol_to_graph(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None: return None
    atoms = [atom_features(atom) for atom in mol.GetAtoms()]
    if not atoms: return None
    x = torch.stack(atoms, dim=0)
    edge_index = []
    for bond in mol.GetBonds():
        i, j = bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()
        edge_index += [[i, j], [j, i]]
    edge_index = torch.tensor(edge_index).t().contiguous()
    edge_attr = [[bond.GetBondTypeAsDouble(), bond.GetIsConjugated(), bond.IsInRing()] * 2 for bond in mol.GetBonds()]
    edge_attr = torch.tensor(edge_attr, dtype=torch.float).reshape(-1, 3)
    return Data(x=x, edge_index=edge_index, edge_attr=edge_attr)

def load_dataset(path, smiles_col='Smiles', target_col='Target'):
    df = pd.read_excel(path)
    graphs = []
    for _, row in df.iterrows():
        if pd.isna(row[smiles_col]) or pd.isna(row[target_col]):
            continue
        g = mol_to_graph(str(row[smiles_col]))
        if g is not None:
            g.y = torch.tensor([row[target_col]], dtype=torch.float)
            g.smiles = row[smiles_col]
            graphs.append(g)
    return graphs

# Import TransformerGCN architecture from cell1
class MultiHeadAttention(torch.nn.Module):
    def __init__(self, hidden_dim, num_heads, dropout=0.1):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.num_heads = num_heads
        
        # Ensure hidden_dim is divisible by num_heads
        # If not, we'll adjust the head_dim to make it work
        self.head_dim = hidden_dim // num_heads
        
        # Actual dimension after ensuring divisibility
        self.actual_dim = self.head_dim * num_heads
        
        self.query = torch.nn.Linear(hidden_dim, self.actual_dim)
        self.key = torch.nn.Linear(hidden_dim, self.actual_dim)
        self.value = torch.nn.Linear(hidden_dim, self.actual_dim)
        self.proj = torch.nn.Linear(self.actual_dim, hidden_dim)
        self.dropout = torch.nn.Dropout(dropout)
        
    def forward(self, x):
        batch_size = x.size(0)
        seq_length = x.size(1)
        
        # Linear projections
        q = self.query(x).view(batch_size, seq_length, self.num_heads, self.head_dim).transpose(1, 2)
        k = self.key(x).view(batch_size, seq_length, self.num_heads, self.head_dim).transpose(1, 2)
        v = self.value(x).view(batch_size, seq_length, self.num_heads, self.head_dim).transpose(1, 2)
        
        # Scaled dot-product attention
        scores = torch.matmul(q, k.transpose(-2, -1)) / (self.head_dim ** 0.5)
        
        # Apply attention weights
        attn_weights = F.softmax(scores, dim=-1)
        attn_weights = self.dropout(attn_weights)
        context = torch.matmul(attn_weights, v)
        
        # Concatenate heads and put through final linear layer
        context = context.transpose(1, 2).contiguous().view(batch_size, seq_length, self.actual_dim)
        output = self.proj(context)
        
        return output

# Transformer Block
class TransformerBlock(torch.nn.Module):
    def __init__(self, hidden_dim, num_heads, ff_dim, dropout=0.1):
        super().__init__()
        self.attention = MultiHeadAttention(hidden_dim, num_heads, dropout)
        self.norm1 = torch.nn.LayerNorm(hidden_dim)
        self.norm2 = torch.nn.LayerNorm(hidden_dim)
        self.ff = torch.nn.Sequential(
            torch.nn.Linear(hidden_dim, ff_dim),
            torch.nn.GELU(),
            torch.nn.Dropout(dropout),
            torch.nn.Linear(ff_dim, hidden_dim),
            torch.nn.Dropout(dropout)
        )
        
    def forward(self, x):
        # Self attention + residual connection
        attn_output = self.attention(x)
        x = self.norm1(x + attn_output)
        
        # Feed forward + residual connection
        ff_output = self.ff(x)
        x = self.norm2(x + ff_output)
        
        return x

# TransformerGCN model
class TransformerGCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers=3, 
                 num_heads=4, ff_dim_factor=4, dropout=0.1):
        super().__init__()
        self.embedding = torch.nn.Linear(in_channels, hidden_channels)
        
        # GCN Layers
        self.convs = torch.nn.ModuleList()
        for _ in range(num_layers):
            self.convs.append(GCNConv(hidden_channels, hidden_channels))
        
        # Transformer blocks (one after each GCN layer)
        self.transformers = torch.nn.ModuleList()
        for _ in range(num_layers):
            self.transformers.append(
                TransformerBlock(hidden_channels, num_heads, hidden_channels * ff_dim_factor, dropout)
            )
        
        # Output layers
        self.fc1 = torch.nn.Linear(hidden_channels, hidden_channels // 2)
        self.fc2 = torch.nn.Linear(hidden_channels // 2, out_channels)
        self.dropout = dropout
        
        # Save for reshaping in forward
        self.hidden_channels = hidden_channels
        self.num_layers = num_layers

    def forward(self, x, edge_index, edge_attr, batch):
        # Initial embedding
        x = self.embedding(x)
        
        # Apply GCN and Transformer layers
        for i, (conv, transformer) in enumerate(zip(self.convs, self.transformers)):
            # GCN Layer
            x_conv = conv(x, edge_index)
            x_conv = F.relu(x_conv)
            x_conv = F.dropout(x_conv, p=self.dropout, training=self.training)
            
            # Reshape for transformer (using a safer approach)
            # Create a padded tensor per batch
            batch_idx = torch.unique(batch)
            transformed_features = []
            
            for b in batch_idx:
                # Get nodes for this batch
                mask = (batch == b)
                nodes_batch = x_conv[mask]
                
                # Process through transformer (add batch dimension)
                if nodes_batch.size(0) > 0:  # Skip empty batches
                    # Add batch and sequence dimensions
                    nodes_batch = nodes_batch.unsqueeze(0)
                    
                    # Apply transformer
                    transformed = transformer(nodes_batch)
                    
                    # Remove batch dimension
                    transformed = transformed.squeeze(0)
                    transformed_features.append(transformed)
            
            # Check if we have any valid features
            if transformed_features:
                # Concatenate all transformed features
                x = torch.cat(transformed_features, dim=0)
            else:
                # Fallback - shouldn't typically happen
                x = x_conv
        
        # Global pooling
        x = global_mean_pool(x, batch)
        
        # MLP for final prediction
        x = self.fc1(x)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.fc2(x)
        
        return x

# Main execution function for generating confusion matrix
def generate_confusion_matrix():
    # Use the correct output directory from cell1 for TransformerGCN
    output_dir = 'Transformer_GCN'
    os.makedirs(output_dir, exist_ok=True)
    
    # Load the best parameters for TransformerGCN
    try:
        with open(os.path.join(output_dir, "study_best_params_transformer_gcn.json"), "r") as f:
            best_params = json.load(f)
        print("Best parameters loaded successfully!")
    except FileNotFoundError:
        print("Best parameters file not found. Using default parameters.")
        best_params = {
            "hidden_channels": 64,
            "num_layers": 3,
            "num_heads": 4,
            "ff_dim_factor": 4,
            "dropout": 0.1,
            "lr": 0.001
        }
    
    # Load dataset and create test loader
    graphs = load_dataset("dataset_main.xlsx")
    from sklearn.model_selection import train_test_split
    _, test_data = train_test_split(graphs, test_size=0.2, random_state=42)
    test_loader = DataLoader(test_data, batch_size=32)
    
    # Initialize TransformerGCN model with best parameters
    model = TransformerGCN(
        20, 
        best_params["hidden_channels"], 
        1, 
        best_params["num_layers"], 
        best_params["num_heads"],
        best_params["ff_dim_factor"], 
        best_params["dropout"]
    )
    
    # Load trained model weights
    try:
        model.load_state_dict(torch.load(os.path.join(output_dir, "transformer_gcn_best_model.pth")))
        print("Model loaded successfully!")
    except FileNotFoundError:
        print("Model file not found. Cannot generate confusion matrix without trained model.")
        return
    
    # Generate predictions
    model.eval()
    y_true = []
    y_pred = []
    y_probs = []
    molecule_smiles = []
    
    with torch.no_grad():
        for data in test_loader:
            out = model(data.x, data.edge_index, data.edge_attr, data.batch).squeeze()
            probs = torch.sigmoid(out).cpu().numpy()
            preds = (probs > 0.5).astype(int)
            
            y_probs.extend(probs)
            y_pred.extend(preds)
            y_true.extend(data.y.cpu().numpy())
            molecule_smiles.extend([g.smiles for g in data.to_data_list()])
    
    # Create confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    class_names = ['Negative', 'Positive']
    
    # Plot confusion matrix
    plt.figure(figsize=(10, 8))
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
    disp.plot(cmap=plt.cm.Blues, values_format='d')
    plt.title('Confusion Matrix for TransformerGCN Model')
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, "confusion_matrix.png"))
    
    # Create a prettier confusion matrix with seaborn
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.title('Confusion Matrix for TransformerGCN Model')
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, "confusion_matrix_seaborn.png"))
    
    # Create a dataframe with predictions for analysis
    results_df = pd.DataFrame({
        'SMILES': molecule_smiles,
        'True_Label': y_true,
        'Predicted_Label': y_pred,
        'Prediction_Probability': y_probs
    })
    
    # Save predictions to Excel
    results_df.to_excel(os.path.join(output_dir, "transformer_gcn_prediction_results.xlsx"), index=False)
    
    # Calculate metrics for each class
    TN, FP, FN, TP = cm.ravel()
    
    # True Positive Rate (Sensitivity or Recall)
    TPR = TP / (TP + FN) if (TP + FN) > 0 else 0
    
    # True Negative Rate (Specificity)
    TNR = TN / (TN + FP) if (TN + FP) > 0 else 0
    
    # False Positive Rate
    FPR = FP / (FP + TN) if (FP + TN) > 0 else 0
    
    # False Negative Rate
    FNR = FN / (TP + FN) if (TP + FN) > 0 else 0
    
    # Precision
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    
    # F1 Score
    f1 = 2 * TP / (2 * TP + FP + FN) if (2 * TP + FP + FN) > 0 else 0
    
    # Print detailed metrics
    print("\n===== Confusion Matrix =====")
    print(f"True Negative (TN): {TN}")
    print(f"False Positive (FP): {FP}")
    print(f"False Negative (FN): {FN}")
    print(f"True Positive (TP): {TP}")
    print("\n===== Detailed Metrics =====")
    print(f"True Positive Rate (Sensitivity/Recall): {TPR:.4f}")
    print(f"True Negative Rate (Specificity): {TNR:.4f}")
    print(f"False Positive Rate: {FPR:.4f}")
    print(f"False Negative Rate: {FNR:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"F1 Score: {f1:.4f}")
    
    # Save metrics to file
    metrics_df = pd.DataFrame({
        'Metric': ['TN', 'FP', 'FN', 'TP', 'Sensitivity/Recall', 'Specificity', 
                  'False Positive Rate', 'False Negative Rate', 'Precision', 'F1 Score'],
        'Value': [TN, FP, FN, TP, TPR, TNR, FPR, FNR, precision, f1]
    })
    metrics_df.to_excel(os.path.join(output_dir, "transformer_gcn_detailed_metrics.xlsx"), index=False)
    
    return results_df

# Run the function
if __name__ == "__main__":
    generate_confusion_matrix()

# External Dataset Evaluation 

## Phytos_Results.xlsx
- Predicted Results saved to excel
- ROC plot and Confusion Matrix plot

In [ ]:
# Cell 2: External Dataset Evaluation with Transformer GCN Model

# Import additional required libraries
import seaborn as sns
from sklearn.exceptions import UndefinedMetricWarning
import warnings

# Ensure output directory exists
output_dir = 'Transformer_GCN'
os.makedirs(output_dir, exist_ok=True)

# First, examine the saved model to understand its architecture
saved_state_dict = torch.load(os.path.join(output_dir, "transformer_gcn_best_model.pth"))

# Print the keys to see the model structure
print("Saved model layers:")
for key in saved_state_dict.keys():
    if 'weight' in key:
        print(f"{key}: {saved_state_dict[key].shape}")

# Function to load external dataset
def load_external_dataset(path, smiles_col='Smiles', target_col='Target'):
    df = pd.read_excel(path)
    graphs = []
    molecules = []
    
    for _, row in df.iterrows():
        if pd.isna(row[smiles_col]):
            continue
            
        g = mol_to_graph(str(row[smiles_col]))
        if g is not None:
            # If target is available, use it, otherwise set to None
            if target_col in df.columns and not pd.isna(row[target_col]):
                g.y = torch.tensor([row[target_col]], dtype=torch.float)
            else:
                g.y = None
                
            g.smiles = row[smiles_col]
            graphs.append(g)
            molecules.append(row.to_dict())
    
    return graphs, pd.DataFrame(molecules)

# Custom model class that matches the saved TransformerGCN architecture
class TransformerGCNForLoading(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers=3, 
                 num_heads=4, ff_dim_factor=4, dropout=0.1):
        super().__init__()
        self.embedding = torch.nn.Linear(in_channels, hidden_channels)
        
        # GCN Layers
        self.convs = torch.nn.ModuleList()
        for _ in range(num_layers):
            self.convs.append(GCNConv(hidden_channels, hidden_channels))
        
        # Transformer blocks (one after each GCN layer)
        self.transformers = torch.nn.ModuleList()
        for _ in range(num_layers):
            self.transformers.append(
                TransformerBlock(hidden_channels, num_heads, hidden_channels * ff_dim_factor, dropout)
            )
        
        # Output layers
        self.fc1 = torch.nn.Linear(hidden_channels, hidden_channels // 2)
        self.fc2 = torch.nn.Linear(hidden_channels // 2, out_channels)
        self.dropout = dropout
        
        # Save for reshaping in forward
        self.hidden_channels = hidden_channels
        self.num_layers = num_layers

    def forward(self, x, edge_index, edge_attr, batch):
        # Initial embedding
        x = self.embedding(x)
        
        # Apply GCN and Transformer layers
        for i, (conv, transformer) in enumerate(zip(self.convs, self.transformers)):
            # GCN Layer
            x_conv = conv(x, edge_index)
            x_conv = F.relu(x_conv)
            x_conv = F.dropout(x_conv, p=self.dropout, training=self.training)
            
            # Reshape for transformer (using a safer approach)
            # Create a padded tensor per batch
            batch_idx = torch.unique(batch)
            transformed_features = []
            
            for b in batch_idx:
                # Get nodes for this batch
                mask = (batch == b)
                nodes_batch = x_conv[mask]
                
                # Process through transformer (add batch dimension)
                if nodes_batch.size(0) > 0:  # Skip empty batches
                    # Add batch and sequence dimensions
                    nodes_batch = nodes_batch.unsqueeze(0)
                    
                    # Apply transformer
                    transformed = transformer(nodes_batch)
                    
                    # Remove batch dimension
                    transformed = transformed.squeeze(0)
                    transformed_features.append(transformed)
            
            # Check if we have any valid features
            if transformed_features:
                # Concatenate all transformed features
                x = torch.cat(transformed_features, dim=0)
            else:
                # Fallback - shouldn't typically happen
                x = x_conv
        
        # Global pooling
        x = global_mean_pool(x, batch)
        
        # MLP for final prediction
        x = self.fc1(x)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.fc2(x)
        
        return x

# Determine model parameters from saved state dict
# We need to load the best hyperparameters from the JSON file
try:
    best_params = json.load(open(os.path.join(output_dir, "study_best_params_transformer_gcn.json"), "r"))
    hidden_channels = best_params["hidden_channels"]
    num_layers = best_params["num_layers"]
    num_heads = best_params["num_heads"]
    ff_dim_factor = best_params["ff_dim_factor"]
    dropout = best_params["dropout"]
    print(f"Loaded parameters: hidden_channels={hidden_channels}, num_layers={num_layers}, "
          f"num_heads={num_heads}, ff_dim_factor={ff_dim_factor}, dropout={dropout}")
except Exception as e:
    print(f"Error loading hyperparameters: {str(e)}")
    # Default values if JSON loading fails
    hidden_channels = 64
    num_layers = 3
    num_heads = 4
    ff_dim_factor = 4
    dropout = 0.1
    print(f"Using default parameters: hidden_channels={hidden_channels}, num_layers={num_layers}, "
          f"num_heads={num_heads}, ff_dim_factor={ff_dim_factor}, dropout={dropout}")

# Create the model with the correct architecture
model = TransformerGCNForLoading(
    in_channels=20,  # Using 20 input features as in Cell 1
    hidden_channels=hidden_channels, 
    out_channels=1,
    num_layers=num_layers,
    num_heads=num_heads,
    ff_dim_factor=ff_dim_factor,
    dropout=dropout
)

# Try to load the saved state
try:
    model.load_state_dict(torch.load(os.path.join(output_dir, "transformer_gcn_best_model.pth")))
    print("Successfully loaded model weights")
except Exception as e:
    print(f"Error loading model weights: {str(e)}")
    print("This may happen if the model architecture doesn't match the saved weights.")
    print("Make sure you've trained the model with the updated architecture first.")
    # Continue anyway for demonstration purposes
    print("Continuing with uninitialized model for demonstration...")

# Set to evaluation mode
model.eval()

# Load external dataset
external_data, external_df = load_external_dataset("Phytos_results.xlsx")
# Suppress deprecation warning for DataLoader
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=UserWarning)
    external_loader = DataLoader(external_data, batch_size=32)

# Get predictions
y_true = []
y_probs = []
smiles_list = []

with torch.no_grad():
    for batch in external_loader:
        try:
            out = model(batch.x, batch.edge_index, batch.edge_attr, batch.batch)
            # Don't squeeze - handle the dimensionality properly
            probs = torch.sigmoid(out).cpu().numpy()
            
            # Handle both single-item and multi-item batches
            if len(probs.shape) == 0:  # Single item case (0-d array)
                probs = np.array([probs.item()])
            
            for i in range(len(probs)):
                y_probs.append(probs[i][0] if probs[i].size > 1 else probs[i])
                
                # Get the SMILES safely
                if hasattr(batch, 'smiles'):
                    if isinstance(batch.smiles, list):
                        smiles_list.append(batch.smiles[i])
                    else:
                        # Handle the case where smiles might be a single string
                        smiles_list.append(batch.smiles)
                
                # Get the true value if it exists
                if hasattr(batch, 'y') and batch.y is not None:
                    if batch.y.dim() > 0 and i < batch.y.size(0):
                        y_true.append(batch.y[i].item())
                    elif batch.y.dim() == 0:  # Single item
                        y_true.append(batch.y.item())
        except Exception as e:
            print(f"Error processing batch: {str(e)}")
            continue

# Save predictions to Excel
results_df = pd.DataFrame({
    'Smiles': smiles_list,
    'Predicted_Probability': y_probs,
    'Predicted_Class': [1 if p > 0.5 else 0 for p in y_probs]
})

# If we have true values, add them and calculate metrics
if len(y_true) > 0 and len(y_true) == len(y_probs):
    results_df['Actual'] = y_true
    
    # Calculate metrics
    y_true_np = np.array(y_true)  # Convert list to numpy array
    y_probs_np = np.array(y_probs)
    y_preds = (y_probs_np > 0.5).astype(int)
    
    # Check if we have multiple classes for metrics that require it
    unique_classes = np.unique(y_true_np)
    has_multiple_classes = len(unique_classes) > 1
    
    metrics = {
        'Accuracy': accuracy_score(y_true_np, y_preds),
        'Precision': precision_score(y_true_np, y_preds, zero_division=0),
        'F1': f1_score(y_true_np, y_preds, zero_division=0),
        'Kappa': cohen_kappa_score(y_true_np, y_preds),
        'Brier': brier_score_loss(y_true_np, y_probs_np),
    }
    
    # Only calculate AUC and MCC if we have multiple classes
    if has_multiple_classes:
        # Suppress the UndefinedMetricWarning
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", category=UndefinedMetricWarning)
            metrics['AUC'] = roc_auc_score(y_true_np, y_probs_np)
            metrics['MCC'] = matthews_corrcoef(y_true_np, y_preds)
    else:
        print("\n⚠️ Warning: Only one class is present in the dataset.")
        print("AUC and MCC are not defined in this case and will be reported as N/A.")
        metrics['AUC'] = "N/A"
        metrics['MCC'] = "N/A"
    
    # Convert N/A values to NaN for Excel export
    metrics_for_excel = {k: np.nan if v == "N/A" else v for k, v in metrics.items()}
    
    # Save metrics
    pd.DataFrame([metrics_for_excel]).to_excel(os.path.join(output_dir, "external_metrics.xlsx"), index=False)
    
    print("\n🧪 External Dataset Metrics:")
    for key, value in metrics.items():
        if isinstance(value, str):
            print(f"{key}: {value}")
        else:
            print(f"{key}: {value:.4f}")
    
    # Only generate ROC curve if we have multiple classes
    if has_multiple_classes:
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", category=UndefinedMetricWarning)
            fpr, tpr, _ = roc_curve(y_true_np, y_probs_np)
            
        plt.figure(figsize=(10, 8))
        plt.plot(fpr, tpr, label=f"AUC = {metrics['AUC']:.2f}" if isinstance(metrics['AUC'], float) else "AUC = N/A")
        plt.plot([0, 1], [0, 1], '--', color='gray')
        plt.xlabel("False Positive Rate")
        plt.ylabel("True Positive Rate")
        plt.title("External Dataset ROC Curve")
        plt.legend()
        plt.savefig(os.path.join(output_dir, "external_roc.png"))
        plt.close()
    
    # Create confusion matrix
    cm = confusion_matrix(y_true_np, y_preds)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.title('Confusion Matrix - External Dataset')
    plt.savefig(os.path.join(output_dir, "external_confusion_matrix.png"))
    plt.close()

    # Class distribution information
    class_counts = np.bincount(y_true_np.astype(int))
    print("\n📊 Class Distribution in External Dataset:")
    for i, count in enumerate(class_counts):
        print(f"Class {i}: {count} samples ({count/len(y_true_np)*100:.1f}%)")

# Save all predictions
results_df.to_excel(os.path.join(output_dir, "external_predictions.xlsx"), index=False)
print(f"\n💾 Predictions for {len(results_df)} molecules saved to {os.path.join(output_dir, 'external_predictions.xlsx')}")

## Cancer_set.xlsx
- Predicted Results saved to excel
- ROC plot 
- Confusion Matrix plot

In [ ]:
# Cell 3: External Dataset Evaluation with Transformer GCN Model

# Import additional required libraries
import seaborn as sns
from sklearn.exceptions import UndefinedMetricWarning
import warnings

# Ensure output directory exists
output_dir = 'Transformer_GCN'
os.makedirs(output_dir, exist_ok=True)

# First, examine the saved model to understand its architecture
saved_state_dict = torch.load(os.path.join(output_dir, "transformer_gcn_best_model.pth"))

# Print the keys to see the model structure
print("Saved model layers:")
for key in saved_state_dict.keys():
    if 'weight' in key:
        print(f"{key}: {saved_state_dict[key].shape}")

# Function to load external dataset
def load_external_dataset(path, smiles_col='Smiles', target_col='Target'):
    df = pd.read_excel(path)
    graphs = []
    molecules = []
    
    for _, row in df.iterrows():
        if pd.isna(row[smiles_col]):
            continue
            
        g = mol_to_graph(str(row[smiles_col]))
        if g is not None:
            # If target is available, use it, otherwise set to None
            if target_col in df.columns and not pd.isna(row[target_col]):
                g.y = torch.tensor([row[target_col]], dtype=torch.float)
            else:
                g.y = None
                
            g.smiles = row[smiles_col]
            graphs.append(g)
            molecules.append(row.to_dict())
    
    return graphs, pd.DataFrame(molecules)

# Custom model class that matches the saved TransformerGCN architecture
class TransformerGCNForLoading(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers=3, 
                 num_heads=4, ff_dim_factor=4, dropout=0.1):
        super().__init__()
        self.embedding = torch.nn.Linear(in_channels, hidden_channels)
        
        # GCN Layers
        self.convs = torch.nn.ModuleList()
        for _ in range(num_layers):
            self.convs.append(GCNConv(hidden_channels, hidden_channels))
        
        # Transformer blocks (one after each GCN layer)
        self.transformers = torch.nn.ModuleList()
        for _ in range(num_layers):
            self.transformers.append(
                TransformerBlock(hidden_channels, num_heads, hidden_channels * ff_dim_factor, dropout)
            )
        
        # Output layers
        self.fc1 = torch.nn.Linear(hidden_channels, hidden_channels // 2)
        self.fc2 = torch.nn.Linear(hidden_channels // 2, out_channels)
        self.dropout = dropout
        
        # Save for reshaping in forward
        self.hidden_channels = hidden_channels
        self.num_layers = num_layers

    def forward(self, x, edge_index, edge_attr, batch):
        # Initial embedding
        x = self.embedding(x)
        
        # Apply GCN and Transformer layers
        for i, (conv, transformer) in enumerate(zip(self.convs, self.transformers)):
            # GCN Layer
            x_conv = conv(x, edge_index)
            x_conv = F.relu(x_conv)
            x_conv = F.dropout(x_conv, p=self.dropout, training=self.training)
            
            # Reshape for transformer (using a safer approach)
            # Create a padded tensor per batch
            batch_idx = torch.unique(batch)
            transformed_features = []
            
            for b in batch_idx:
                # Get nodes for this batch
                mask = (batch == b)
                nodes_batch = x_conv[mask]
                
                # Process through transformer (add batch dimension)
                if nodes_batch.size(0) > 0:  # Skip empty batches
                    # Add batch and sequence dimensions
                    nodes_batch = nodes_batch.unsqueeze(0)
                    
                    # Apply transformer
                    transformed = transformer(nodes_batch)
                    
                    # Remove batch dimension
                    transformed = transformed.squeeze(0)
                    transformed_features.append(transformed)
            
            # Check if we have any valid features
            if transformed_features:
                # Concatenate all transformed features
                x = torch.cat(transformed_features, dim=0)
            else:
                # Fallback - shouldn't typically happen
                x = x_conv
        
        # Global pooling
        x = global_mean_pool(x, batch)
        
        # MLP for final prediction
        x = self.fc1(x)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.fc2(x)
        
        return x

# Determine model parameters from saved state dict
# We need to load the best hyperparameters from the JSON file
try:
    best_params = json.load(open(os.path.join(output_dir, "study_best_params_transformer_gcn.json"), "r"))
    hidden_channels = best_params["hidden_channels"]
    num_layers = best_params["num_layers"]
    num_heads = best_params["num_heads"]
    ff_dim_factor = best_params["ff_dim_factor"]
    dropout = best_params["dropout"]
    print(f"Loaded parameters: hidden_channels={hidden_channels}, num_layers={num_layers}, "
          f"num_heads={num_heads}, ff_dim_factor={ff_dim_factor}, dropout={dropout}")
except Exception as e:
    print(f"Error loading hyperparameters: {str(e)}")
    # Default values if JSON loading fails
    hidden_channels = 64
    num_layers = 3
    num_heads = 4
    ff_dim_factor = 4
    dropout = 0.1
    print(f"Using default parameters: hidden_channels={hidden_channels}, num_layers={num_layers}, "
          f"num_heads={num_heads}, ff_dim_factor={ff_dim_factor}, dropout={dropout}")

# Create the model with the correct architecture
model = TransformerGCNForLoading(
    in_channels=20,  # Using 20 input features as in Cell 1
    hidden_channels=hidden_channels, 
    out_channels=1,
    num_layers=num_layers,
    num_heads=num_heads,
    ff_dim_factor=ff_dim_factor,
    dropout=dropout
)

# Try to load the saved state
try:
    model.load_state_dict(torch.load(os.path.join(output_dir, "transformer_gcn_best_model.pth")))
    print("Successfully loaded model weights")
except Exception as e:
    print(f"Error loading model weights: {str(e)}")
    print("This may happen if the model architecture doesn't match the saved weights.")
    print("Make sure you've trained the model with the updated architecture first.")
    # Continue anyway for demonstration purposes
    print("Continuing with uninitialized model for demonstration...")

# Set to evaluation mode
model.eval()

# Load external dataset
external_data, external_df = load_external_dataset("cancer_set.xlsx")
# Suppress deprecation warning for DataLoader
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=UserWarning)
    external_loader = DataLoader(external_data, batch_size=32)

# Get predictions
y_true = []
y_probs = []
smiles_list = []

with torch.no_grad():
    for batch in external_loader:
        try:
            out = model(batch.x, batch.edge_index, batch.edge_attr, batch.batch)
            # Don't squeeze - handle the dimensionality properly
            probs = torch.sigmoid(out).cpu().numpy()
            
            # Handle both single-item and multi-item batches
            if len(probs.shape) == 0:  # Single item case (0-d array)
                probs = np.array([probs.item()])
            
            for i in range(len(probs)):
                y_probs.append(probs[i][0] if probs[i].size > 1 else probs[i])
                
                # Get the SMILES safely
                if hasattr(batch, 'smiles'):
                    if isinstance(batch.smiles, list):
                        smiles_list.append(batch.smiles[i])
                    else:
                        # Handle the case where smiles might be a single string
                        smiles_list.append(batch.smiles)
                
                # Get the true value if it exists
                if hasattr(batch, 'y') and batch.y is not None:
                    if batch.y.dim() > 0 and i < batch.y.size(0):
                        y_true.append(batch.y[i].item())
                    elif batch.y.dim() == 0:  # Single item
                        y_true.append(batch.y.item())
        except Exception as e:
            print(f"Error processing batch: {str(e)}")
            continue

# Save predictions to Excel
results_df = pd.DataFrame({
    'Smiles': smiles_list,
    'Predicted_Probability': y_probs,
    'Predicted_Class': [1 if p > 0.5 else 0 for p in y_probs]
})

# If we have true values, add them and calculate metrics
if len(y_true) > 0 and len(y_true) == len(y_probs):
    results_df['Actual'] = y_true
    
    # Calculate metrics
    y_true_np = np.array(y_true)  # Convert list to numpy array
    y_probs_np = np.array(y_probs)
    y_preds = (y_probs_np > 0.5).astype(int)
    
    # Check if we have multiple classes for metrics that require it
    unique_classes = np.unique(y_true_np)
    has_multiple_classes = len(unique_classes) > 1
    
    metrics = {
        'Accuracy': accuracy_score(y_true_np, y_preds),
        'Precision': precision_score(y_true_np, y_preds, zero_division=0),
        'F1': f1_score(y_true_np, y_preds, zero_division=0),
        'Kappa': cohen_kappa_score(y_true_np, y_preds),
        'Brier': brier_score_loss(y_true_np, y_probs_np),
    }
    
    # Only calculate AUC and MCC if we have multiple classes
    if has_multiple_classes:
        # Suppress the UndefinedMetricWarning
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", category=UndefinedMetricWarning)
            metrics['AUC'] = roc_auc_score(y_true_np, y_probs_np)
            metrics['MCC'] = matthews_corrcoef(y_true_np, y_preds)
    else:
        print("\n⚠️ Warning: Only one class is present in the dataset.")
        print("AUC and MCC are not defined in this case and will be reported as N/A.")
        metrics['AUC'] = "N/A"
        metrics['MCC'] = "N/A"
    
    # Convert N/A values to NaN for Excel export
    metrics_for_excel = {k: np.nan if v == "N/A" else v for k, v in metrics.items()}
    
    # Save metrics
    pd.DataFrame([metrics_for_excel]).to_excel(os.path.join(output_dir, "external_metrics_cancer.xlsx"), index=False)
    
    print("\n🧪 External Dataset Metrics:")
    for key, value in metrics.items():
        if isinstance(value, str):
            print(f"{key}: {value}")
        else:
            print(f"{key}: {value:.4f}")
    
    # Only generate ROC curve if we have multiple classes
    if has_multiple_classes:
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", category=UndefinedMetricWarning)
            fpr, tpr, _ = roc_curve(y_true_np, y_probs_np)
            
        plt.figure(figsize=(10, 8))
        plt.plot(fpr, tpr, label=f"AUC = {metrics['AUC']:.2f}" if isinstance(metrics['AUC'], float) else "AUC = N/A")
        plt.plot([0, 1], [0, 1], '--', color='gray')
        plt.xlabel("False Positive Rate")
        plt.ylabel("True Positive Rate")
        plt.title("External Dataset ROC Curve")
        plt.legend()
        plt.savefig(os.path.join(output_dir, "external_roc_cancer.png"))
        plt.close()
    
    # Create confusion matrix
    cm = confusion_matrix(y_true_np, y_preds)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.title('Confusion Matrix - External Dataset')
    plt.savefig(os.path.join(output_dir, "external_cancer_confusion_matrix.png"))
    plt.close()

    # Class distribution information
    class_counts = np.bincount(y_true_np.astype(int))
    print("\n📊 Class Distribution in External Dataset:")
    for i, count in enumerate(class_counts):
        print(f"Class {i}: {count} samples ({count/len(y_true_np)*100:.1f}%)")

# Save all predictions
results_df.to_excel(os.path.join(output_dir, "external_predictions_cancer.xlsx"), index=False)
print(f"\n💾 Predictions for {len(results_df)} molecules saved to {os.path.join(output_dir, 'external_predictions_cancer.xlsx')}")

# Explinibilty using Integrated Gradients

##  Dataset used cancer_set.xlsx(FDA approved)
- Top 40 most contributing SMILES plot
- Feature Importance plot and saved the values in xlsx
- Node importance vlaue saved in xlsx
- Atom Type Importance plot

In [ ]:
import os
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from rdkit import Chem
from rdkit.Chem import Draw, AllChem
from rdkit.Chem.Draw import rdMolDraw2D
from torch_geometric.data import DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool
import seaborn as sns
from captum.attr import IntegratedGradients
from tqdm import tqdm
import shutil
import json

# Create output directory
output_dir = os.path.join('Transformer_GCN', 'external_transformer_gcn_cancer')
os.makedirs(output_dir, exist_ok=True)

# Import TransformerBlock and MultiHeadAttention from original code
class MultiHeadAttention(torch.nn.Module):
    def __init__(self, hidden_dim, num_heads, dropout=0.1):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.num_heads = num_heads
        
        # Ensure hidden_dim is divisible by num_heads
        self.head_dim = hidden_dim // num_heads
        self.actual_dim = self.head_dim * num_heads
        
        self.query = torch.nn.Linear(hidden_dim, self.actual_dim)
        self.key = torch.nn.Linear(hidden_dim, self.actual_dim)
        self.value = torch.nn.Linear(hidden_dim, self.actual_dim)
        self.proj = torch.nn.Linear(self.actual_dim, hidden_dim)
        self.dropout = torch.nn.Dropout(dropout)
        
    def forward(self, x):
        batch_size = x.size(0)
        seq_length = x.size(1)
        
        # Linear projections
        q = self.query(x).view(batch_size, seq_length, self.num_heads, self.head_dim).transpose(1, 2)
        k = self.key(x).view(batch_size, seq_length, self.num_heads, self.head_dim).transpose(1, 2)
        v = self.value(x).view(batch_size, seq_length, self.num_heads, self.head_dim).transpose(1, 2)
        
        # Scaled dot-product attention
        scores = torch.matmul(q, k.transpose(-2, -1)) / (self.head_dim ** 0.5)
        
        # Apply attention weights
        attn_weights = torch.nn.functional.softmax(scores, dim=-1)
        attn_weights = self.dropout(attn_weights)
        context = torch.matmul(attn_weights, v)
        
        # Concatenate heads and put through final linear layer
        context = context.transpose(1, 2).contiguous().view(batch_size, seq_length, self.actual_dim)
        output = self.proj(context)
        
        return output

class TransformerBlock(torch.nn.Module):
    def __init__(self, hidden_dim, num_heads, ff_dim, dropout=0.1):
        super().__init__()
        self.attention = MultiHeadAttention(hidden_dim, num_heads, dropout)
        self.norm1 = torch.nn.LayerNorm(hidden_dim)
        self.norm2 = torch.nn.LayerNorm(hidden_dim)
        self.ff = torch.nn.Sequential(
            torch.nn.Linear(hidden_dim, ff_dim),
            torch.nn.GELU(),
            torch.nn.Dropout(dropout),
            torch.nn.Linear(ff_dim, hidden_dim),
            torch.nn.Dropout(dropout)
        )
        
    def forward(self, x):
        # Self attention + residual connection
        attn_output = self.attention(x)
        x = self.norm1(x + attn_output)
        
        # Feed forward + residual connection
        ff_output = self.ff(x)
        x = self.norm2(x + ff_output)
        
        return x

# Define the TransformerGCN model class (needed for loading the model)
class TransformerGCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers=3, 
                 num_heads=4, ff_dim_factor=4, dropout=0.1):
        super().__init__()
        self.embedding = torch.nn.Linear(in_channels, hidden_channels)
        
        # GCN Layers
        self.convs = torch.nn.ModuleList()
        for _ in range(num_layers):
            self.convs.append(GCNConv(hidden_channels, hidden_channels))
        
        # Transformer blocks (one after each GCN layer)
        self.transformers = torch.nn.ModuleList()
        for _ in range(num_layers):
            self.transformers.append(
                TransformerBlock(hidden_channels, num_heads, hidden_channels * ff_dim_factor, dropout)
            )
        
        # Output layers
        self.fc1 = torch.nn.Linear(hidden_channels, hidden_channels // 2)
        self.fc2 = torch.nn.Linear(hidden_channels // 2, out_channels)
        self.dropout = dropout
        
        # Save for reshaping in forward
        self.hidden_channels = hidden_channels
        self.num_layers = num_layers

    def forward(self, x, edge_index, edge_attr, batch):
        # Initial embedding
        x = self.embedding(x)
        
        # Apply GCN and Transformer layers
        for i, (conv, transformer) in enumerate(zip(self.convs, self.transformers)):
            # GCN Layer
            x_conv = conv(x, edge_index)
            x_conv = torch.nn.functional.relu(x_conv)
            x_conv = torch.nn.functional.dropout(x_conv, p=self.dropout, training=self.training)
            
            # Reshape for transformer (using a safer approach)
            batch_idx = torch.unique(batch)
            transformed_features = []
            
            for b in batch_idx:
                # Get nodes for this batch
                mask = (batch == b)
                nodes_batch = x_conv[mask]
                
                # Process through transformer (add batch dimension)
                if nodes_batch.size(0) > 0:  # Skip empty batches
                    # Add batch and sequence dimensions
                    nodes_batch = nodes_batch.unsqueeze(0)
                    
                    # Apply transformer
                    transformed = transformer(nodes_batch)
                    
                    # Remove batch dimension
                    transformed = transformed.squeeze(0)
                    transformed_features.append(transformed)
            
            # Check if we have any valid features
            if transformed_features:
                # Concatenate all transformed features
                x = torch.cat(transformed_features, dim=0)
            else:
                # Fallback - shouldn't typically happen
                x = x_conv
        
        # Global pooling
        x = global_mean_pool(x, batch)
        
        # MLP for final prediction
        x = self.fc1(x)
        x = torch.nn.functional.relu(x)
        x = torch.nn.functional.dropout(x, p=self.dropout, training=self.training)
        x = self.fc2(x)
        
        return x

# Function to convert molecules to graph data
def atom_features(atom):
    return torch.tensor([
        # Basic properties
        atom.GetAtomicNum(),                     # Atomic number
        atom.GetDegree(),                        # Number of bonded neighbors
        atom.GetFormalCharge(),                  # Formal charge
        atom.GetNumRadicalElectrons(),           # Number of radical electrons
        int(atom.GetIsAromatic()),               # Aromaticity flag
        
        # Extended properties
        atom.GetExplicitValence(),               # Explicit valence
        atom.GetImplicitValence(),               # Implicit valence
        atom.GetTotalValence(),                  # Total valence
        atom.GetNumImplicitHs(),                 # Number of implicit hydrogens
        atom.GetHybridization(),                 # Hybridization state
        atom.GetTotalNumHs(),                    # Total number of hydrogens
        
        # Topological properties
        int(atom.IsInRing()),                    # Whether the atom is in a ring
        int(atom.IsInRingSize(3)),               # Whether in 3-membered ring
        int(atom.IsInRingSize(4)),               # Whether in 4-membered ring
        int(atom.IsInRingSize(5)),               # Whether in 5-membered ring
        int(atom.IsInRingSize(6)),               # Whether in 6-membered ring
        int(atom.IsInRingSize(7)),               # Whether in 7-membered ring
        
        # Electronic properties
        atom.GetChiralTag(),                     # Chirality
        atom.GetMass(),                          # Atomic mass
        Chem.rdMolDescriptors.CalcCrippenDescriptors(
            Chem.MolFromSmiles(f"[{atom.GetSymbol()}]"))[0]  # LogP contribution
    ], dtype=torch.float)

def mol_to_graph(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None: return None
    atoms = [atom_features(atom) for atom in mol.GetAtoms()]
    if not atoms: return None
    x = torch.stack(atoms, dim=0)
    edge_index = []
    for bond in mol.GetBonds():
        i, j = bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()
        edge_index += [[i, j], [j, i]]
    edge_index = torch.tensor(edge_index).t().contiguous()
    edge_attr = [[bond.GetBondTypeAsDouble(), bond.GetIsConjugated(), bond.IsInRing()] * 2 for bond in mol.GetBonds()]
    edge_attr = torch.tensor(edge_attr, dtype=torch.float).reshape(-1, 3)
    from torch_geometric.data import Data
    return Data(x=x, edge_index=edge_index, edge_attr=edge_attr)

def load_dataset(path, smiles_col='Smiles', target_col='Target'):
    df = pd.read_excel(path)
    graphs = []
    for _, row in df.iterrows():
        if pd.isna(row[smiles_col]) or pd.isna(row[target_col]):
            continue
        g = mol_to_graph(str(row[smiles_col]))
        if g is not None:
            g.y = torch.tensor([row[target_col]], dtype=torch.float)
            g.smiles = row[smiles_col]
            graphs.append(g)
    return graphs

# Load the trained model
def load_trained_model(model_path, input_dim=20, hidden_params_path=os.path.join('Transformer_GCN', 'study_best_params_transformer_gcn.json')):
    try:
        with open(hidden_params_path, 'r') as f:
            best_params = json.load(f)
        
        model = TransformerGCN(
            in_channels=input_dim,
            hidden_channels=best_params['hidden_channels'],
            out_channels=1, 
            num_layers=best_params['num_layers'], 
            num_heads=best_params['num_heads'],
            ff_dim_factor=best_params['ff_dim_factor'], 
            dropout=best_params['dropout']
        )
    except FileNotFoundError:
        # Fallback to default parameters if file not found
        print(f"Parameters file not found at {hidden_params_path}. Using default parameters.")
        model = TransformerGCN(
            in_channels=input_dim,
            hidden_channels=64,
            out_channels=1,
            num_layers=3,
            num_heads=4,
            ff_dim_factor=4,
            dropout=0.1
        )
    
    model.load_state_dict(torch.load(model_path))
    model.eval()
    return model

# Load model
model_path = os.path.join('Transformer_GCN', 'transformer_gcn_best_model.pth')
model = load_trained_model(model_path)

# Load dataset
train_data = load_dataset("cancer_set.xlsx")  # Load your full dataset

# Helper function to create a wrapper model for attribution methods
class TransformerGCNWrapper(torch.nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model
        
    def forward(self, x, edge_index, edge_attr, batch):
        return torch.sigmoid(self.model(x, edge_index, edge_attr, batch))

# Initialize attribution method
wrapper_model = TransformerGCNWrapper(model)
integrated_gradients = IntegratedGradients(wrapper_model)

# Function to interpret molecule graph and get node/edge importance
def interpret_molecule(graph):
    graph_batch = graph.clone()
    graph_batch.batch = torch.zeros(graph.x.shape[0], dtype=torch.long)
    
    # Get integrated gradients attribution
    node_attr = integrated_gradients.attribute(
        graph.x, 
        additional_forward_args=(graph.edge_index, graph.edge_attr, graph_batch.batch),
        internal_batch_size=1
    )
    
    # Sum across feature dimensions to get per-node importance
    node_importance = torch.abs(node_attr).sum(dim=1).detach().numpy()
    
    # For edge importance, we'll use a simple approximation based on connected nodes
    edge_importance = []
    for i in range(graph.edge_index.shape[1]):
        src, dst = graph.edge_index[0, i].item(), graph.edge_index[1, i].item()
        edge_imp = (node_importance[src] + node_importance[dst]) / 2
        edge_importance.append(edge_imp)
    
    return node_importance, np.array(edge_importance)

# Function to create a safe filename from SMILES
def create_safe_filename(smiles):
    filename = f"{smiles}_transformer_gcn.png"
    # Replace characters that are problematic in filenames
    for char in ['/', '\\', ':', '*', '?', '"', '<', '>', '|']:
        filename = filename.replace(char, '_')
    # Ensure the filename isn't too long for the filesystem
    if len(filename) > 200:
        # Truncate but keep the suffix
        filename = filename[:185] + "_tgcn_cancer.png"
    return filename

# Function to visualize molecule with importance highlighting
def visualize_molecule_importance(smiles, node_importance, mol=None, title=None, size=(600, 600)):
    if mol is None:
        mol = Chem.MolFromSmiles(smiles)
    
    # Normalize importances for coloring
    if len(node_importance) > 0:
        norm = Normalize(vmin=np.min(node_importance), vmax=np.max(node_importance))
        atom_colors = {}
        for i, imp in enumerate(node_importance):
            color_val = norm(imp)
            atom_colors[i] = tuple(plt.cm.viridis(color_val)[:3])
    else:
        atom_colors = {}
    
    # Create drawing
    drawer = rdMolDraw2D.MolDraw2DCairo(size[0], size[1])
    drawer.SetFontSize(0.8)
    
    # Set up atom highlights
    highlight_atoms = list(range(mol.GetNumAtoms()))
    highlight_atom_colors = atom_colors
    
    # Add title if provided
    if title:
        drawer.DrawMoleculeWithHighlights(
            mol, title, 
            highlightAtoms=highlight_atoms,
            highlightAtomColors=highlight_atom_colors
        )
    else:
        # Draw the molecule with highlights
        drawer.DrawMolecule(
            mol,
            highlightAtoms=highlight_atoms,
            highlightAtomColors=highlight_atom_colors
        )
    drawer.FinishDrawing()
    
    return drawer.GetDrawingText()

# Function to analyze feature importance across the dataset
def analyze_feature_importance(data_list, top_n=20):
    all_attributions = []
    
    for graph in tqdm(data_list, desc="Analyzing feature importance"):
        graph_batch = graph.clone()
        graph_batch.batch = torch.zeros(graph.x.shape[0], dtype=torch.long)
        
        attr = integrated_gradients.attribute(
            graph.x, 
            additional_forward_args=(graph.edge_index, graph.edge_attr, graph_batch.batch),
            internal_batch_size=1
        )
        
        # Average attribution per feature across all nodes
        avg_attr = torch.abs(attr).mean(dim=0).detach().numpy()
        all_attributions.append(avg_attr)
    
    # Average across all molecules
    feature_importance = np.mean(all_attributions, axis=0)
    
    # Feature names (same as in atom_features function)
    feature_names = [
        'Atomic number', 'Degree', 'Formal charge', 'Radical electrons', 'Is aromatic',
        'Explicit valence', 'Implicit valence', 'Total valence', 'Implicit Hs',
        'Hybridization', 'Total Hs', 'In ring', 'In ring size 3', 'In ring size 4',
        'In ring size 5', 'In ring size 6', 'In ring size 7', 'Chiral tag', 'Mass',
        'LogP contribution'
    ]
    
    # Sort by importance
    importance_df = pd.DataFrame({
        'Feature': feature_names,
        'Importance': feature_importance
    }).sort_values('Importance', ascending=False)
    
    return importance_df

# Create output subfolders
viz_dir = os.path.join(output_dir, 'molecule_viz_cancer')
os.makedirs(viz_dir, exist_ok=True)

top_molecules_dir = os.path.join(output_dir, 'top_molecules')
os.makedirs(top_molecules_dir, exist_ok=True)

# Process all molecules in the dataset
molecule_importances = []

# Process a subset of molecules (limit to 100 for efficiency)
num_molecules = min(100, len(train_data))

for i, graph in enumerate(tqdm(train_data[:num_molecules], desc="Processing molecules")):
    try:
        smiles = graph.smiles
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            continue
            
        # Get prediction score
        graph_batch = graph.clone()
        graph_batch.batch = torch.zeros(graph.x.shape[0], dtype=torch.long)
        pred_score = wrapper_model(graph.x, graph.edge_index, graph.edge_attr, graph_batch.batch).item()
        
        # Get node and edge importance
        node_importance, edge_importance = interpret_molecule(graph)
        
        # Calculate importance metrics
        avg_importance = np.mean(node_importance)
        max_importance = np.max(node_importance)
        
        # Create a safe filename from the SMILES
        safe_filename = create_safe_filename(smiles)
        
        # Save molecule image with importance visualization
        img_data = visualize_molecule_importance(smiles, node_importance, mol)
        with open(os.path.join(viz_dir, safe_filename), "wb") as f:
            f.write(img_data)
        
        # Add to our results dataframe
        molecule_importances.append({
            'Index': i+1,
            'SMILES': smiles,
            'Prediction': pred_score,
            'True_Label': graph.y.item(),
            'Average_Importance': avg_importance,
            'Max_Importance': max_importance,
            'Most_Important_Atom_Index': np.argmax(node_importance),
            'Filename': safe_filename
        })
    except Exception as e:
        print(f"Error processing molecule {i+1}: {e}")

# Create and save dataframe of molecule importances
importance_df = pd.DataFrame(molecule_importances)
importance_df.to_excel(os.path.join(output_dir, 'molecule_importances_cancer.xlsx'), index=False)

# Create feature importance analysis
feature_imp_df = analyze_feature_importance(train_data[:num_molecules])
feature_imp_df.to_excel(os.path.join(output_dir, 'feature_importance_cancer.xlsx'), index=False)

# Plot and save feature importance graph
plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=feature_imp_df.head(15))
plt.title('Top 15 Important Atom Features (TransformerGCN Analysis)')
plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'feature_importance_cancer.png'), dpi=300)
plt.close()

# Find the top 40 molecules by importance
top_molecules_count = min(40, len(importance_df))
top_mols = importance_df.sort_values('Average_Importance', ascending=False).head(top_molecules_count)

# Create a top SMILES CSV file
with open(os.path.join(output_dir, "top_smiles_cancer.csv"), "w") as f:
    f.write("Rank,MoleculeIndex,SMILES,Prediction,TrueLabel,AverageImportance,Filename\n")
    for j, (_, r) in enumerate(top_mols.iterrows()):
        f.write(f"{j+1},{r['Index']},{r['SMILES']},{r['Prediction']:.4f},{r['True_Label']:.0f},{r['Average_Importance']:.4f},{r['Filename']}\n")

# Create a higher-quality version of the top molecules (with titles)
for i, (_, row) in enumerate(tqdm(top_mols.iterrows(), desc="Creating high-quality top molecule images")):
    try:
        # Get the molecule and its data
        idx = row['Index']
        smiles = row['SMILES']
        pred = row['Prediction']
        true_label = row['True_Label']
        
        # Get the corresponding graph data
        graph = train_data[idx-1]  # -1 because indices start at 1 in our data
        
        # Re-interpret the molecule for visualization
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            continue
            
        # Get node importance
        node_importance, _ = interpret_molecule(graph)
        
        # Create a title with info
        title = f"#{idx}: Pred={pred:.3f}, True={true_label:.0f}, Importance={row['Average_Importance']:.3f}"
        
        # Create a high-quality visualization
        img_data = visualize_molecule_importance(smiles, node_importance, mol, title=title, size=(800, 800))
        
        # Use the same filename as before
        safe_filename = row['Filename']
        
        # Save to the top molecules directory
        with open(os.path.join(top_molecules_dir, safe_filename), "wb") as f:
            f.write(img_data)
            
    except Exception as e:
        print(f"Error creating top molecule image for rank {i+1}: {e}")

# Create node importance analysis
node_importances = []
for i, graph in enumerate(tqdm(train_data[:num_molecules], desc="Analyzing node types")):
    try:
        smiles = graph.smiles
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            continue
            
        node_importance, _ = interpret_molecule(graph)
        
        # Get atom types and their importance
        for atom_idx, importance in enumerate(node_importance):
            atom_symbol = mol.GetAtomWithIdx(atom_idx).GetSymbol()
            node_importances.append({
                'Molecule_Index': i+1,
                'SMILES': smiles,
                'Atom_Index': atom_idx,
                'Atom_Symbol': atom_symbol,
                'Importance': importance
            })
    except Exception as e:
        print(f"Error analyzing node for molecule {i+1}: {e}")

# Create and save node importance dataframe
node_imp_df = pd.DataFrame(node_importances)
node_imp_df.to_excel(os.path.join(output_dir, 'node_importances_cancer.xlsx'), index=False)

# Analyze importance by atom type
atom_importance = node_imp_df.groupby('Atom_Symbol')['Importance'].mean().reset_index()
atom_importance = atom_importance.sort_values('Importance', ascending=False)
atom_importance.to_excel(os.path.join(output_dir, 'atom_type_importance_cancer.xlsx'), index=False)

# Plot atom type importance
plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Atom_Symbol', data=atom_importance)
plt.title('Average Importance by Atom Type (TransformerGCN Analysis)')
plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'atom_type_importance_cancer.png'), dpi=300)
plt.close()

# Create a color-coded Excel file with molecule importances
try:
    from openpyxl import load_workbook
    from openpyxl.styles import PatternFill
    from openpyxl.styles.differential import DifferentialStyle
    from openpyxl.formatting.rule import ColorScaleRule
    
    color_excel_path = os.path.join(output_dir, 'molecule_importances_colored_cancer.xlsx')
    importance_df.to_excel(color_excel_path, index=False)
    
    # Open the workbook and add color scales
    wb = load_workbook(color_excel_path)
    ws = wb.active
    
    # Add color scale to Average_Importance column
    col_idx = importance_df.columns.get_loc("Average_Importance") + 1  # +1 because Excel is 1-indexed
    col_letter = ws.cell(1, col_idx).column_letter
    
    ws.conditional_formatting.add(
        f"{col_letter}2:{col_letter}{len(importance_df)+1}",
        ColorScaleRule(
            start_type='min', start_color='FFFFFF',
            mid_type='percentile', mid_value=50, mid_color='FFFF00',
            end_type='max', end_color='FF0000'
        )
    )
    
    # Add color scale to Max_Importance column
    col_idx = importance_df.columns.get_loc("Max_Importance") + 1
    col_letter = ws.cell(1, col_idx).column_letter
    
    ws.conditional_formatting.add(
        f"{col_letter}2:{col_letter}{len(importance_df)+1}",
        ColorScaleRule(
            start_type='min', start_color='FFFFFF',
            mid_type='percentile', mid_value=50, mid_color='FFFF00',
            end_type='max', end_color='FF0000'
        )
    )
    
    # Save the workbook
    wb.save(color_excel_path)
    print(f"Created color-coded Excel file: {color_excel_path}")
except ImportError:
    print("openpyxl not available - skipping colored Excel creation")

print(f"✅ Analysis complete! All results saved to {output_dir}")
print(f"📊 Key files generated:")
print(f" - Molecule visualizations with SMILES filenames: {viz_dir}/*.png")
print(f" - Top 40 molecules: {top_molecules_dir}/*.png")
print(f" - Top molecules SMILES list: {os.path.join(output_dir, 'top_smiles_cancer.csv')}")
print(f" - Feature importance analysis: {os.path.join(output_dir, 'feature_importance_cancer.xlsx')}")
print(f" - Molecule importance spreadsheet: {os.path.join(output_dir, 'molecule_importances_cancer.xlsx')}")
print(f" - Atom type importance: {os.path.join(output_dir, 'atom_type_importance_cancer.png')}")

##  Dataset used dataset_main.xlsx
- Top 40 most contributing SMILES plot
- Feature Importance plot and saved the values in xlsx
- Node importance vlaue saved in xlsx
- Atom Type Importance plot

In [ ]:
import os
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from rdkit import Chem
from rdkit.Chem import Draw, AllChem
from rdkit.Chem.Draw import rdMolDraw2D
from torch_geometric.data import DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool
import seaborn as sns
from captum.attr import IntegratedGradients
from tqdm import tqdm
import shutil
import json

# Create output directory
output_dir = os.path.join('Transformer_GCN', 'external_transformer_gcn')
os.makedirs(output_dir, exist_ok=True)

# Import TransformerBlock and MultiHeadAttention from original code
class MultiHeadAttention(torch.nn.Module):
    def __init__(self, hidden_dim, num_heads, dropout=0.1):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.num_heads = num_heads
        
        # Ensure hidden_dim is divisible by num_heads
        self.head_dim = hidden_dim // num_heads
        self.actual_dim = self.head_dim * num_heads
        
        self.query = torch.nn.Linear(hidden_dim, self.actual_dim)
        self.key = torch.nn.Linear(hidden_dim, self.actual_dim)
        self.value = torch.nn.Linear(hidden_dim, self.actual_dim)
        self.proj = torch.nn.Linear(self.actual_dim, hidden_dim)
        self.dropout = torch.nn.Dropout(dropout)
        
    def forward(self, x):
        batch_size = x.size(0)
        seq_length = x.size(1)
        
        # Linear projections
        q = self.query(x).view(batch_size, seq_length, self.num_heads, self.head_dim).transpose(1, 2)
        k = self.key(x).view(batch_size, seq_length, self.num_heads, self.head_dim).transpose(1, 2)
        v = self.value(x).view(batch_size, seq_length, self.num_heads, self.head_dim).transpose(1, 2)
        
        # Scaled dot-product attention
        scores = torch.matmul(q, k.transpose(-2, -1)) / (self.head_dim ** 0.5)
        
        # Apply attention weights
        attn_weights = torch.nn.functional.softmax(scores, dim=-1)
        attn_weights = self.dropout(attn_weights)
        context = torch.matmul(attn_weights, v)
        
        # Concatenate heads and put through final linear layer
        context = context.transpose(1, 2).contiguous().view(batch_size, seq_length, self.actual_dim)
        output = self.proj(context)
        
        return output

class TransformerBlock(torch.nn.Module):
    def __init__(self, hidden_dim, num_heads, ff_dim, dropout=0.1):
        super().__init__()
        self.attention = MultiHeadAttention(hidden_dim, num_heads, dropout)
        self.norm1 = torch.nn.LayerNorm(hidden_dim)
        self.norm2 = torch.nn.LayerNorm(hidden_dim)
        self.ff = torch.nn.Sequential(
            torch.nn.Linear(hidden_dim, ff_dim),
            torch.nn.GELU(),
            torch.nn.Dropout(dropout),
            torch.nn.Linear(ff_dim, hidden_dim),
            torch.nn.Dropout(dropout)
        )
        
    def forward(self, x):
        # Self attention + residual connection
        attn_output = self.attention(x)
        x = self.norm1(x + attn_output)
        
        # Feed forward + residual connection
        ff_output = self.ff(x)
        x = self.norm2(x + ff_output)
        
        return x

# Define the TransformerGCN model class (needed for loading the model)
class TransformerGCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers=3, 
                 num_heads=4, ff_dim_factor=4, dropout=0.1):
        super().__init__()
        self.embedding = torch.nn.Linear(in_channels, hidden_channels)
        
        # GCN Layers
        self.convs = torch.nn.ModuleList()
        for _ in range(num_layers):
            self.convs.append(GCNConv(hidden_channels, hidden_channels))
        
        # Transformer blocks (one after each GCN layer)
        self.transformers = torch.nn.ModuleList()
        for _ in range(num_layers):
            self.transformers.append(
                TransformerBlock(hidden_channels, num_heads, hidden_channels * ff_dim_factor, dropout)
            )
        
        # Output layers
        self.fc1 = torch.nn.Linear(hidden_channels, hidden_channels // 2)
        self.fc2 = torch.nn.Linear(hidden_channels // 2, out_channels)
        self.dropout = dropout
        
        # Save for reshaping in forward
        self.hidden_channels = hidden_channels
        self.num_layers = num_layers

    def forward(self, x, edge_index, edge_attr, batch):
        # Initial embedding
        x = self.embedding(x)
        
        # Apply GCN and Transformer layers
        for i, (conv, transformer) in enumerate(zip(self.convs, self.transformers)):
            # GCN Layer
            x_conv = conv(x, edge_index)
            x_conv = torch.nn.functional.relu(x_conv)
            x_conv = torch.nn.functional.dropout(x_conv, p=self.dropout, training=self.training)
            
            # Reshape for transformer (using a safer approach)
            batch_idx = torch.unique(batch)
            transformed_features = []
            
            for b in batch_idx:
                # Get nodes for this batch
                mask = (batch == b)
                nodes_batch = x_conv[mask]
                
                # Process through transformer (add batch dimension)
                if nodes_batch.size(0) > 0:  # Skip empty batches
                    # Add batch and sequence dimensions
                    nodes_batch = nodes_batch.unsqueeze(0)
                    
                    # Apply transformer
                    transformed = transformer(nodes_batch)
                    
                    # Remove batch dimension
                    transformed = transformed.squeeze(0)
                    transformed_features.append(transformed)
            
            # Check if we have any valid features
            if transformed_features:
                # Concatenate all transformed features
                x = torch.cat(transformed_features, dim=0)
            else:
                # Fallback - shouldn't typically happen
                x = x_conv
        
        # Global pooling
        x = global_mean_pool(x, batch)
        
        # MLP for final prediction
        x = self.fc1(x)
        x = torch.nn.functional.relu(x)
        x = torch.nn.functional.dropout(x, p=self.dropout, training=self.training)
        x = self.fc2(x)
        
        return x

# Function to convert molecules to graph data
def atom_features(atom):
    return torch.tensor([
        # Basic properties
        atom.GetAtomicNum(),                     # Atomic number
        atom.GetDegree(),                        # Number of bonded neighbors
        atom.GetFormalCharge(),                  # Formal charge
        atom.GetNumRadicalElectrons(),           # Number of radical electrons
        int(atom.GetIsAromatic()),               # Aromaticity flag
        
        # Extended properties
        atom.GetExplicitValence(),               # Explicit valence
        atom.GetImplicitValence(),               # Implicit valence
        atom.GetTotalValence(),                  # Total valence
        atom.GetNumImplicitHs(),                 # Number of implicit hydrogens
        atom.GetHybridization(),                 # Hybridization state
        atom.GetTotalNumHs(),                    # Total number of hydrogens
        
        # Topological properties
        int(atom.IsInRing()),                    # Whether the atom is in a ring
        int(atom.IsInRingSize(3)),               # Whether in 3-membered ring
        int(atom.IsInRingSize(4)),               # Whether in 4-membered ring
        int(atom.IsInRingSize(5)),               # Whether in 5-membered ring
        int(atom.IsInRingSize(6)),               # Whether in 6-membered ring
        int(atom.IsInRingSize(7)),               # Whether in 7-membered ring
        
        # Electronic properties
        atom.GetChiralTag(),                     # Chirality
        atom.GetMass(),                          # Atomic mass
        Chem.rdMolDescriptors.CalcCrippenDescriptors(
            Chem.MolFromSmiles(f"[{atom.GetSymbol()}]"))[0]  # LogP contribution
    ], dtype=torch.float)

def mol_to_graph(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None: return None
    atoms = [atom_features(atom) for atom in mol.GetAtoms()]
    if not atoms: return None
    x = torch.stack(atoms, dim=0)
    edge_index = []
    for bond in mol.GetBonds():
        i, j = bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()
        edge_index += [[i, j], [j, i]]
    edge_index = torch.tensor(edge_index).t().contiguous()
    edge_attr = [[bond.GetBondTypeAsDouble(), bond.GetIsConjugated(), bond.IsInRing()] * 2 for bond in mol.GetBonds()]
    edge_attr = torch.tensor(edge_attr, dtype=torch.float).reshape(-1, 3)
    from torch_geometric.data import Data
    return Data(x=x, edge_index=edge_index, edge_attr=edge_attr)

def load_dataset(path, smiles_col='Smiles', target_col='Target'):
    df = pd.read_excel(path)
    graphs = []
    for _, row in df.iterrows():
        if pd.isna(row[smiles_col]) or pd.isna(row[target_col]):
            continue
        g = mol_to_graph(str(row[smiles_col]))
        if g is not None:
            g.y = torch.tensor([row[target_col]], dtype=torch.float)
            g.smiles = row[smiles_col]
            graphs.append(g)
    return graphs

# Load the trained model
def load_trained_model(model_path, input_dim=20, hidden_params_path=os.path.join('Transformer_GCN', 'study_best_params_transformer_gcn.json')):
    try:
        with open(hidden_params_path, 'r') as f:
            best_params = json.load(f)
        
        model = TransformerGCN(
            in_channels=input_dim,
            hidden_channels=best_params['hidden_channels'],
            out_channels=1, 
            num_layers=best_params['num_layers'], 
            num_heads=best_params['num_heads'],
            ff_dim_factor=best_params['ff_dim_factor'], 
            dropout=best_params['dropout']
        )
    except FileNotFoundError:
        # Fallback to default parameters if file not found
        print(f"Parameters file not found at {hidden_params_path}. Using default parameters.")
        model = TransformerGCN(
            in_channels=input_dim,
            hidden_channels=64,
            out_channels=1,
            num_layers=3,
            num_heads=4,
            ff_dim_factor=4,
            dropout=0.1
        )
    
    model.load_state_dict(torch.load(model_path))
    model.eval()
    return model

# Load model
model_path = os.path.join('Transformer_GCN', 'transformer_gcn_best_model.pth')
model = load_trained_model(model_path)

# Load dataset
train_data = load_dataset("dataset_main.xlsx")  # Load your full dataset

# Helper function to create a wrapper model for attribution methods
class TransformerGCNWrapper(torch.nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model
        
    def forward(self, x, edge_index, edge_attr, batch):
        return torch.sigmoid(self.model(x, edge_index, edge_attr, batch))

# Initialize attribution method
wrapper_model = TransformerGCNWrapper(model)
integrated_gradients = IntegratedGradients(wrapper_model)

# Function to interpret molecule graph and get node/edge importance
def interpret_molecule(graph):
    graph_batch = graph.clone()
    graph_batch.batch = torch.zeros(graph.x.shape[0], dtype=torch.long)
    
    # Get integrated gradients attribution
    node_attr = integrated_gradients.attribute(
        graph.x, 
        additional_forward_args=(graph.edge_index, graph.edge_attr, graph_batch.batch),
        internal_batch_size=1
    )
    
    # Sum across feature dimensions to get per-node importance
    node_importance = torch.abs(node_attr).sum(dim=1).detach().numpy()
    
    # For edge importance, we'll use a simple approximation based on connected nodes
    edge_importance = []
    for i in range(graph.edge_index.shape[1]):
        src, dst = graph.edge_index[0, i].item(), graph.edge_index[1, i].item()
        edge_imp = (node_importance[src] + node_importance[dst]) / 2
        edge_importance.append(edge_imp)
    
    return node_importance, np.array(edge_importance)

# Function to create a safe filename from SMILES
def create_safe_filename(smiles):
    filename = f"{smiles}_transformer_gcn.png"
    # Replace characters that are problematic in filenames
    for char in ['/', '\\', ':', '*', '?', '"', '<', '>', '|']:
        filename = filename.replace(char, '_')
    # Ensure the filename isn't too long for the filesystem
    if len(filename) > 200:
        # Truncate but keep the suffix
        filename = filename[:185] + "_tgcn.png"
    return filename

# Function to visualize molecule with importance highlighting
def visualize_molecule_importance(smiles, node_importance, mol=None, title=None, size=(600, 600)):
    if mol is None:
        mol = Chem.MolFromSmiles(smiles)
    
    # Normalize importances for coloring
    if len(node_importance) > 0:
        norm = Normalize(vmin=np.min(node_importance), vmax=np.max(node_importance))
        atom_colors = {}
        for i, imp in enumerate(node_importance):
            color_val = norm(imp)
            atom_colors[i] = tuple(plt.cm.viridis(color_val)[:3])
    else:
        atom_colors = {}
    
    # Create drawing
    drawer = rdMolDraw2D.MolDraw2DCairo(size[0], size[1])
    drawer.SetFontSize(0.8)
    
    # Set up atom highlights
    highlight_atoms = list(range(mol.GetNumAtoms()))
    highlight_atom_colors = atom_colors
    
    # Add title if provided
    if title:
        drawer.DrawMoleculeWithHighlights(
            mol, title, 
            highlightAtoms=highlight_atoms,
            highlightAtomColors=highlight_atom_colors
        )
    else:
        # Draw the molecule with highlights
        drawer.DrawMolecule(
            mol,
            highlightAtoms=highlight_atoms,
            highlightAtomColors=highlight_atom_colors
        )
    drawer.FinishDrawing()
    
    return drawer.GetDrawingText()

# Function to analyze feature importance across the dataset
def analyze_feature_importance(data_list, top_n=20):
    all_attributions = []
    
    for graph in tqdm(data_list, desc="Analyzing feature importance"):
        graph_batch = graph.clone()
        graph_batch.batch = torch.zeros(graph.x.shape[0], dtype=torch.long)
        
        attr = integrated_gradients.attribute(
            graph.x, 
            additional_forward_args=(graph.edge_index, graph.edge_attr, graph_batch.batch),
            internal_batch_size=1
        )
        
        # Average attribution per feature across all nodes
        avg_attr = torch.abs(attr).mean(dim=0).detach().numpy()
        all_attributions.append(avg_attr)
    
    # Average across all molecules
    feature_importance = np.mean(all_attributions, axis=0)
    
    # Feature names (same as in atom_features function)
    feature_names = [
        'Atomic number', 'Degree', 'Formal charge', 'Radical electrons', 'Is aromatic',
        'Explicit valence', 'Implicit valence', 'Total valence', 'Implicit Hs',
        'Hybridization', 'Total Hs', 'In ring', 'In ring size 3', 'In ring size 4',
        'In ring size 5', 'In ring size 6', 'In ring size 7', 'Chiral tag', 'Mass',
        'LogP contribution'
    ]
    
    # Sort by importance
    importance_df = pd.DataFrame({
        'Feature': feature_names,
        'Importance': feature_importance
    }).sort_values('Importance', ascending=False)
    
    return importance_df

# Create output subfolders
viz_dir = os.path.join(output_dir, 'molecule_viz')
os.makedirs(viz_dir, exist_ok=True)

top_molecules_dir = os.path.join(output_dir, 'top_molecules')
os.makedirs(top_molecules_dir, exist_ok=True)

# Process all molecules in the dataset
molecule_importances = []

# Process a subset of molecules (limit to 100 for efficiency)
num_molecules = min(100, len(train_data))

for i, graph in enumerate(tqdm(train_data[:num_molecules], desc="Processing molecules")):
    try:
        smiles = graph.smiles
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            continue
            
        # Get prediction score
        graph_batch = graph.clone()
        graph_batch.batch = torch.zeros(graph.x.shape[0], dtype=torch.long)
        pred_score = wrapper_model(graph.x, graph.edge_index, graph.edge_attr, graph_batch.batch).item()
        
        # Get node and edge importance
        node_importance, edge_importance = interpret_molecule(graph)
        
        # Calculate importance metrics
        avg_importance = np.mean(node_importance)
        max_importance = np.max(node_importance)
        
        # Create a safe filename from the SMILES
        safe_filename = create_safe_filename(smiles)
        
        # Save molecule image with importance visualization
        img_data = visualize_molecule_importance(smiles, node_importance, mol)
        with open(os.path.join(viz_dir, safe_filename), "wb") as f:
            f.write(img_data)
        
        # Add to our results dataframe
        molecule_importances.append({
            'Index': i+1,
            'SMILES': smiles,
            'Prediction': pred_score,
            'True_Label': graph.y.item(),
            'Average_Importance': avg_importance,
            'Max_Importance': max_importance,
            'Most_Important_Atom_Index': np.argmax(node_importance),
            'Filename': safe_filename
        })
    except Exception as e:
        print(f"Error processing molecule {i+1}: {e}")

# Create and save dataframe of molecule importances
importance_df = pd.DataFrame(molecule_importances)
importance_df.to_excel(os.path.join(output_dir, 'molecule_importances.xlsx'), index=False)

# Create feature importance analysis
feature_imp_df = analyze_feature_importance(train_data[:num_molecules])
feature_imp_df.to_excel(os.path.join(output_dir, 'feature_importance.xlsx'), index=False)

# Plot and save feature importance graph
plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=feature_imp_df.head(15))
plt.title('Top 15 Important Atom Features (TransformerGCN Analysis)')
plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'feature_importance.png'), dpi=300)
plt.close()

# Find the top 40 molecules by importance
top_molecules_count = min(40, len(importance_df))
top_mols = importance_df.sort_values('Average_Importance', ascending=False).head(top_molecules_count)

# Create a top SMILES CSV file
with open(os.path.join(output_dir, "top_smiles_cancer.csv"), "w") as f:
    f.write("Rank,MoleculeIndex,SMILES,Prediction,TrueLabel,AverageImportance,Filename\n")
    for j, (_, r) in enumerate(top_mols.iterrows()):
        f.write(f"{j+1},{r['Index']},{r['SMILES']},{r['Prediction']:.4f},{r['True_Label']:.0f},{r['Average_Importance']:.4f},{r['Filename']}\n")

# Create a higher-quality version of the top molecules (with titles)
for i, (_, row) in enumerate(tqdm(top_mols.iterrows(), desc="Creating high-quality top molecule images")):
    try:
        # Get the molecule and its data
        idx = row['Index']
        smiles = row['SMILES']
        pred = row['Prediction']
        true_label = row['True_Label']
        
        # Get the corresponding graph data
        graph = train_data[idx-1]  # -1 because indices start at 1 in our data
        
        # Re-interpret the molecule for visualization
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            continue
            
        # Get node importance
        node_importance, _ = interpret_molecule(graph)
        
        # Create a title with info
        title = f"#{idx}: Pred={pred:.3f}, True={true_label:.0f}, Importance={row['Average_Importance']:.3f}"
        
        # Create a high-quality visualization
        img_data = visualize_molecule_importance(smiles, node_importance, mol, title=title, size=(800, 800))
        
        # Use the same filename as before
        safe_filename = row['Filename']
        
        # Save to the top molecules directory
        with open(os.path.join(top_molecules_dir, safe_filename), "wb") as f:
            f.write(img_data)
            
    except Exception as e:
        print(f"Error creating top molecule image for rank {i+1}: {e}")

# Create node importance analysis
node_importances = []
for i, graph in enumerate(tqdm(train_data[:num_molecules], desc="Analyzing node types")):
    try:
        smiles = graph.smiles
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            continue
            
        node_importance, _ = interpret_molecule(graph)
        
        # Get atom types and their importance
        for atom_idx, importance in enumerate(node_importance):
            atom_symbol = mol.GetAtomWithIdx(atom_idx).GetSymbol()
            node_importances.append({
                'Molecule_Index': i+1,
                'SMILES': smiles,
                'Atom_Index': atom_idx,
                'Atom_Symbol': atom_symbol,
                'Importance': importance
            })
    except Exception as e:
        print(f"Error analyzing node for molecule {i+1}: {e}")

# Create and save node importance dataframe
node_imp_df = pd.DataFrame(node_importances)
node_imp_df.to_excel(os.path.join(output_dir, 'node_importances.xlsx'), index=False)

# Analyze importance by atom type
atom_importance = node_imp_df.groupby('Atom_Symbol')['Importance'].mean().reset_index()
atom_importance = atom_importance.sort_values('Importance', ascending=False)
atom_importance.to_excel(os.path.join(output_dir, 'atom_type_importance.xlsx'), index=False)

# Plot atom type importance
plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Atom_Symbol', data=atom_importance)
plt.title('Average Importance by Atom Type (TransformerGCN Analysis)')
plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'atom_type_importance.png'), dpi=300)
plt.close()

# Create a color-coded Excel file with molecule importances
try:
    from openpyxl import load_workbook
    from openpyxl.styles import PatternFill
    from openpyxl.styles.differential import DifferentialStyle
    from openpyxl.formatting.rule import ColorScaleRule
    
    color_excel_path = os.path.join(output_dir, 'molecule_importances_colored.xlsx')
    importance_df.to_excel(color_excel_path, index=False)
    
    # Open the workbook and add color scales
    wb = load_workbook(color_excel_path)
    ws = wb.active
    
    # Add color scale to Average_Importance column
    col_idx = importance_df.columns.get_loc("Average_Importance") + 1  # +1 because Excel is 1-indexed
    col_letter = ws.cell(1, col_idx).column_letter
    
    ws.conditional_formatting.add(
        f"{col_letter}2:{col_letter}{len(importance_df)+1}",
        ColorScaleRule(
            start_type='min', start_color='FFFFFF',
            mid_type='percentile', mid_value=50, mid_color='FFFF00',
            end_type='max', end_color='FF0000'
        )
    )
    
    # Add color scale to Max_Importance column
    col_idx = importance_df.columns.get_loc("Max_Importance") + 1
    col_letter = ws.cell(1, col_idx).column_letter
    
    ws.conditional_formatting.add(
        f"{col_letter}2:{col_letter}{len(importance_df)+1}",
        ColorScaleRule(
            start_type='min', start_color='FFFFFF',
            mid_type='percentile', mid_value=50, mid_color='FFFF00',
            end_type='max', end_color='FF0000'
        )
    )
    
    # Save the workbook
    wb.save(color_excel_path)
    print(f"Created color-coded Excel file: {color_excel_path}")
except ImportError:
    print("openpyxl not available - skipping colored Excel creation")

print(f"✅ Analysis complete! All results saved to {output_dir}")
print(f"📊 Key files generated:")
print(f" - Molecule visualizations with SMILES filenames: {viz_dir}/*.png")
print(f" - Top 40 molecules: {top_molecules_dir}/*.png")
print(f" - Top molecules SMILES list: {os.path.join(output_dir, 'top_smiles.csv')}")
print(f" - Feature importance analysis: {os.path.join(output_dir, 'feature_importance.xlsx')}")
print(f" - Molecule importance spreadsheet: {os.path.join(output_dir, 'molecule_importances.xlsx')}")
print(f" - Atom type importance: {os.path.join(output_dir, 'atom_type_importance.png')}")